In [13]:
# Import pandas, numpy, matplotlib, seaborn libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import datetime
import time

In [14]:
dftrips_copy = pd.read_csv('trips_copy.csv')
dftrips_copy

,DAYOFSERVICE,TRIPID,LINEID,ROUTEID,DIRECTION,PLANNEDTIME_ARR,PLANNEDTIME_DEP,ACTUALTIME_ARR,ACTUALTIME_DEP,PLANNED_TRIPTIME(s),ACTUAL_TRIPTIME(s),HOUROFDAY,DAYOFWEEK,MONTHOFYEAR
0,2018-02-07,6253783,68,68_80,1,87245,84600,87524.0,84600.0,2645,2924,23.0,2,2
1,2018-02-07,6262138,25B,25B_271,2,30517,26460,32752.0,NaN,4057,4057,7.0,2,2
2,2018-02-07,6254942,45A,45A_70,2,35512,32100,36329.0,32082.0,3412,4247,9.0,2,2
3,2018-02-07,6259460,25A,25A_273,1,57261,54420,58463.0,54443.0,2841,4020,15.0,2,2
4,2018-02-07,6253175,14,14_15,1,85383,81600,84682.0,81608.0,3783,3074,23.0,2,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2182529,2018-05-14,6765849,123,123_36,2,61560,57840,61365.0,57859.0,3720,3506,16.0,0,5
2182530,2018-05-14,6765469,75,75_17,1,53416,48600,NaN,48823.0,4816,4816,14.0,0,5
2182531,2018-05-14,6765486,33D,33D_62,2,29460,26400,29904.0,NaN,3060,3060,7.0,0,5
2182532,2018-05-14,6764987,70,70_60,1,65277,60600,66341.0,NaN,4677,4677,17.0,0,5


In [15]:
%store -r data
data

'68'

In [22]:
# selecting the tripid where lineid is 46A
tripid_list = dftrips_copy['TRIPID'][dftrips_copy['LINEID'] == data].tolist()
 
# list of dataframes
df_list = []

# iterating over each chunk in the leave times cleaned csv
chunksize = 10 ** 6
for chunk in pd.read_csv("rt_leavetimes_DB_2018.txt", sep=';', chunksize=chunksize):
    # adding the data in the chunk if the trip id is in the list we specified earlier
    df_list.append(chunk[chunk['TRIPID'].isin(tripid_list)])
    
df_concat = pd.concat(df_list)

In [ ]:
#Convert the datatype of dftrips into proper type
dftrips_copy['DAYOFSERVICE'] = dftrips_copy['DAYOFSERVICE'].astype('datetime64')
dftrips_copy['LINEID'] = dftrips_copy['LINEID'].astype('category')
dftrips_copy['ROUTEID'] = dftrips_copy['ROUTEID'].astype('category')

#Convert the datatype into proper type
categorical_columns = df_concat[['TRIPID','DATASOURCE','NOTE']].columns
continuous_columns = df_concat[['PROGRNUMBER','STOPPOINTID','PLANNEDTIME_ARR','PLANNEDTIME_DEP','ACTUALTIME_ARR','ACTUALTIME_DEP','VEHICLEID','PASSENGERS','PASSENGERSIN','PASSENGERSOUT','DISTANCE','SUPPRESSED','JUSTIFICATIONID']].columns
datetime_columns = df_concat[['DAYOFSERVICE', 'LASTUPDATE']].columns

for column in categorical_columns:
    df_concat[column] = df_concat[column].astype('category') 
    
for column in datetime_columns:
    df_concat[column] = df_concat[column].astype('datetime64') 
    
    
# Dropping unnecessary columns
df_concat = df_concat.drop(columns=['DATASOURCE', 'VEHICLEID', 'PASSENGERS','PASSENGERSIN','PASSENGERSOUT', 'DISTANCE', 'SUPPRESSED', 'LASTUPDATE', 'NOTE', 'JUSTIFICATIONID'])

#Merge with the dftrips_copy dataset
df_concat=df_concat.merge(dftrips_copy, on = ['DAYOFSERVICE', 'TRIPID'])

#fill in the null values in ACTUALTIME_DEP_y
for i in range(len(df_concat)):
    if (np.isnan(df_concat["ACTUALTIME_DEP_y"][i])):
        df_concat["ACTUALTIME_DEP_y"][i] = df_concat["PLANNEDTIME_DEP_y"][i]
        
#Calculate the accumulated travel time
df_concat['ACC_TRAVELTIME(s)'] = (df_concat['ACTUALTIME_ARR_x'] - df_concat['ACTUALTIME_DEP_y'])

#ONLY keep the columnd we need
df_concat = df_concat[['DAYOFSERVICE','TRIPID','PROGRNUMBER','STOPPOINTID','LINEID','DIRECTION','PLANNED_TRIPTIME(s)','ACTUAL_TRIPTIME(s)','HOUROFDAY','DAYOFWEEK','MONTHOFYEAR','ACC_TRAVELTIME(s)']]

#Adding the information of 'HOUROFDAY' to the 'DAYOFSERVICE' column
for i in range(len(df_concat['DAYOFSERVICE'])):
    df_concat['DAYOFSERVICE'][i] = df_concat['DAYOFSERVICE'][i] + pd.Timedelta(hours=df_concat['HOUROFDAY'][i])
    
#Importing the weather data
dfweather = pd.read_csv('weather2018_clean.csv')

#Convert data type of dfweather
dfweather['DAYOFSERVICE'] = dfweather['DAYOFSERVICE'].astype('datetime64')
dfweather['weather_main'] = dfweather['weather_main'].astype('category')
dfweather['weather_description'] = dfweather['weather_description'].astype('category')

# merging data with weather data (merge on date and hour)
df_concat = df_concat.merge(dfweather, on=['DAYOFSERVICE'])

In [ ]:
#Fill in the blank with 0.
#df_concat['visibility'].fillna(0, inplace=True)
#df_concat['wind_gust'].fillna(0, inplace=True)
#df_concat['rain_1h'].fillna(0, inplace=True)
#df_concat['snow_1h'].fillna(0, inplace=True)

# list trip and dayofservice for all trips with outlier data (4 times above mean travel time or below 0 travel time)
outliers = df_concat.loc[(df_concat['ACC_TRAVELTIME(s)'] > df_concat['ACC_TRAVELTIME(s)'].mean()*4) | (df_concat['ACC_TRAVELTIME(s)'] < 0)].index
#drop the outliers
df_concat = df_concat.drop(outliers)

# route 46A trips which took over 10 mins to reach second stop
dfr1longstart = df_concat.loc[(df_concat['ACC_TRAVELTIME(s)'] > 600) & (df_concat['PROGRNUMBER'] == 2)].loc[(df_concat['ACC_TRAVELTIME(s)'] > 600) & (df_concat['PROGRNUMBER'] == 2)]
# index list for rows with over 600 travel time by stop 2
longStartIndexList = df_concat.loc[(df_concat['DAYOFSERVICE'].isin(dfr1longstart['DAYOFSERVICE'].unique())) & (df_concat['TRIPID'].isin(dfr1longstart['TRIPID'].unique()))].index.tolist()
# dropping trips which had over 600 travel time by stop 2
df_concat = df_concat.drop(longStartIndexList)

#Select the features we need for modeling and Copy the original version of dataset
df_concat_ver1 = df_concat[['DIRECTION','PROGRNUMBER','PLANNED_TRIPTIME(s)','HOUROFDAY','DAYOFWEEK','MONTHOFYEAR','ACC_TRAVELTIME(s)','temp','wind_speed','rain_1h']]


# # set up dummies features
# df_concat_ver1 = pd.get_dummies(df_concat_ver1, drop_first=True)
# df_concat_ver1.dtypes

#Setting up the train/test split
# y is the target
y = df_concat_ver1["ACC_TRAVELTIME(s)"]
# X is everything else
X = df_concat_ver1.drop(["ACC_TRAVELTIME(s)"],1)
# Split the dataset into two datasets: 70% training and 30% test
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1,  test_size=0.3)

#Record the time of setting up linear regression model
start_time1 = time.time()
# Train aka fit, a model using all continuous and categorical features.
linreg = LinearRegression().fit(X_train, y_train)

#Predict the test dataset
linreg_predictions_test = linreg.predict(X_test)
    
#Calculate the metrics
MAE_linreg = metrics.mean_absolute_error(y_test, linreg_predictions_test)
RMSE_linreg = metrics.mean_squared_error(y_test, linreg_predictions_test)**0.5
R2_linreg = metrics.r2_score(y_test, linreg_predictions_test)
    
end_time1 = time.time()
timer1 = end_time1 - start_time1
    
# writing these evaluation scores to a file
f = open("LinearRegres_Evaluation_06_08.txt", "a")
f.write("\nCombination of features" + str(i) + ".\n")
f.write("\nLine: " + str(data) + ". MAE: " + str(MAE_linreg) + ". RMSE: " + str(RMSE_linreg) + ". R2 Score: " + str(R2_linreg) + ".\n")
f.write("\nModel running time: " + str(timer1) + ".\n")
f.close()
    
#Record the time of setting up random forest model
start_time2 = time.time()
    
# Train RF with 10 trees
rfc = RandomForestRegressor(n_estimators=10, max_features='auto', oob_score=True, random_state=1).fit(X_train, y_train)

#Predict the test dataset
rfc_predictions_test = rfc.predict(X_test)

#Evaluation scores
MAE_rfc = metrics.mean_absolute_error(y_test, rfc_predictions_test)
RMSE_rfc = metrics.mean_squared_error(y_test, rfc_predictions_test)**0.5
R2_rfc = metrics.r2_score(y_test, rfc_predictions_test)
    
end_time2 = time.time()
timer2 = end_time2 - start_time2
    
# writing these evaluation scores to a file
f = open("RandomForest_Evaluation_06_08.txt", "a")
f.write("\nCombination of features" + str(i) + ".\n")
f.write("\nLine: " + str(data) + ". MAE: " + str(MAE_rfc) + ". RMSE: " + str(RMSE_rfc) + ". R2 Score: " + str(R2_rfc) + ".\n")
f.write("\nModel running time: " + str(timer2) + ".\n")
f.close()

# save the model to pickle file
filename = "ModelResult_LinearRegression/PickleFiles_{}.pkl".format(lineid)
pickle.dump(linreg, open(filename, 'wb'))

# save the model to pickle file
filename = "ModelResult_RandomForest/PickleFiles_{}.pkl".format(lineid)
pickle.dump(rfc, open(filename, 'wb'))
